In [2]:
!pip install -q accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatib

##Used only for preprocess initial dataset

In [3]:
import pandas as pd
import re

# Function to split the text into Human and Assistant sections
def split_text(text):
    # Split the text based on the start of the Assistant section
    parts = re.split(r"###Assistant:", text, maxsplit=1)

    # If split results in more than 1 part, handle the sections
    if len(parts) > 1:
        human_text = parts[0].strip()
        assistant_text = "###Assistant:" + parts[1].strip()
    else:
        # If no Assistant section is found, handle accordingly
        human_text = text.strip()
        assistant_text = ""

    return human_text, assistant_text

# Read the CSV file
input_csv = '/content/train.csv'  # Replace with your file path
df = pd.read_csv(input_csv)

# Create lists to hold the split texts
human_texts = []
assistant_texts = []

# Process each row
for index, row in df.iterrows():
    human_text, assistant_text = split_text(row['text'])  # Replace 'text_column' with the actual column name
    human_texts.append(human_text)
    assistant_texts.append(assistant_text)

# Create a new DataFrame
new_df = pd.DataFrame({
    'Human': human_texts,
    'Assistant': assistant_texts
})

# Save to a new CSV file
output_csv = 'dental.csv'
new_df.to_csv(output_csv, index=False)


KeyError: 'text'

##Prepare the Dataset

In [4]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/dental.csv')
df = df[:100]

# Convert the DataFrame into a Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

# Split dataset into train and validation sets
dataset_split = dataset.train_test_split(test_size=0.2)
train_dataset = dataset_split['train']
valid_dataset = dataset_split['test']

system_message = "This is a dental radiology assistant model. Answer the questions based on the findings."

# Preprocess the dataset
def preprocess_function(examples):
    # Access each column in the examples
    return {
        'text': [
            f"[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n{human} [/INST] {assistant}"
            for human, assistant in zip(examples['Human'], examples['Assistant'])
        ]
    }

# Apply the preprocessing function to both datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

##Fine-tune the Model

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
# import torch
# from tqdm import tqdm

# # Model and tokenizer setup
# model_name = "NousResearch/llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # LoRA configuration
# lora_config = LoraConfig(
#     r=64,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# # Training arguments
# training_args = {
#     "output_dir": "./results",
#     "num_train_epochs": 3,
#     "per_device_train_batch_size": 4,
#     "gradient_accumulation_steps": 1,
#     "learning_rate": 2e-4,
#     "weight_decay": 0.001,
#     "logging_steps": 5,
#     "save_steps": 25,
#     "evaluation_strategy": "steps",
#     "eval_steps": 25,
# }

# # Prepare the model with LoRA configuration
# model = PeftModel.from_pretrained(model, lora_config)

# # Initialize SFTTrainer
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_dataset,
#     eval_dataset=valid_dataset,
#     peft_config=lora_config,
#     tokenizer=tokenizer,
#     args=training_args
# )


from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import torch
from tqdm import tqdm

# Model and tokenizer setup
model_name = "NousResearch/llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA configuration
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA configuration
model = PeftModel(model=model, lora_config=lora_config)

# Training arguments with optimized settings
training_args = {
    "output_dir": "./results",
    "num_train_epochs": 3,
    "per_device_train_batch_size": 2,  # Reduced batch size to fit memory constraints
    "gradient_accumulation_steps": 4,  # Accumulate gradients over multiple steps
    "learning_rate": 2e-4,
    "weight_decay": 0.001,
    "logging_steps": 5,
    "save_steps": 25,
    "evaluation_strategy": "steps",
    "eval_steps": 25,
    "fp16": True,  # Use mixed precision to reduce memory usage
}

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    args=training_args
)

# Define a custom training loop with tqdm
def train_with_tqdm(trainer):
    num_training_steps = len(train_dataset) // training_args['per_device_train_batch_size']
    progress_bar = tqdm(range(num_training_steps), desc="Training Progress")

    for _ in trainer.train():
        progress_bar.update(1)

    trainer.model.save_pretrained("./llama-2-7b-dental")

# Run the custom training loop
train_with_tqdm(trainer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Define a custom training loop with tqdm
def train_with_tqdm(trainer):
    # Start training
    num_training_steps = len(train_dataset) // training_args['per_device_train_batch_size']
    progress_bar = tqdm(range(num_training_steps), desc="Training Progress")

    for _ in trainer.train():
        progress_bar.update(1)

    # Save the fine-tuned model
    trainer.model.save_pretrained("./llama-2-7b-dental")

# Run the custom training loop
train_with_tqdm(trainer)

##Deploy to Huggingface

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(folder_path="./llama-2-7b-dental", repo_id="Chanvitha/llama-2-7b-dental")